<a href="https://colab.research.google.com/github/isabelklint/scrapers/blob/main/Ethiopian_reporter_WEB_SCRAPER_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://github.com/tilburgsciencehub/data-ranking-the-brands/blob/main/webscraping_ranking_the_brands.ipynb
!pip install BeautifulSoup4
!pip install pandas
!pip install requests

from bs4 import BeautifulSoup
import requests, lxml
import pandas as pd


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# categories do not include drop-downs
# categories = ['politics', 'business', 'editorial', 'kibur-minister', 'sport', 'kinin-and-bahil', 'opinion', 'temuaget', 'speak-your-mind', 'world' ]
# https://www.ethiopianreporter.com

# GET SITEMAPS

sitemaps = []

URL = 'https://www.ethiopianreporter.com/sitemap_index.xml'
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"
headers = {"user-agent": USER_AGENT}
resp = requests.get(URL, headers=headers)

soup = BeautifulSoup(resp.content, "xml")
locs = soup.find_all("loc")

for loc in locs:
    sitemaps.append((loc.text))


# <?xml version="1.0" encoding="UTF-8"?><?xml-stylesheet type="text/xsl" href="//www.ethiopianreporter.com/main-sitemap.xsl"?><html><body><sitemapindex xmlns="http://www.sitemaps.org/schemas/sitemap/0.9">
# <sitemap>
# <loc>https://www.ethiopianreporter.com/post-sitemap1.xml</loc>
# <lastmod>2023-02-07T07:53:11+00:00</lastmod>
# </sitemap>


In [ ]:
# print(sitemaps)

In [ ]:
len(sitemaps)

In [4]:
# make some lists
list_of_URLS = []
scrapedURLS = []


In [5]:
# get URLS from each page of sitemaps
for sitemap in sitemaps:
  URL = sitemap
  USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"
  headers = {"user-agent": USER_AGENT}
  resp = requests.get(URL, headers=headers)

  soup = BeautifulSoup(resp.content, "xml")
  locs = soup.find_all("loc")

  for loc in locs:
      list_of_URLS.append((loc.text))


In [6]:
print(list_of_URLS)

['https://www.ethiopianreporter.com/115495/', 'https://www.ethiopianreporter.com/115492/', 'https://www.ethiopianreporter.com/115390/', 'https://www.ethiopianreporter.com/115402/', 'https://www.ethiopianreporter.com/115468/', 'https://www.ethiopianreporter.com/115463/', 'https://www.ethiopianreporter.com/115457/', 'https://www.ethiopianreporter.com/115460/', 'https://www.ethiopianreporter.com/115455/', 'https://www.ethiopianreporter.com/115453/', 'https://www.ethiopianreporter.com/115451/', 'https://www.ethiopianreporter.com/115450/', 'https://www.ethiopianreporter.com/115442/', 'https://www.ethiopianreporter.com/115439/', 'https://www.ethiopianreporter.com/115432/', 'https://www.ethiopianreporter.com/115435/', 'http://www.ethiopianreporter.com/wp-content/uploads/2023/01/Brieff.jpg', 'https://www.ethiopianreporter.com/115429/', 'https://www.ethiopianreporter.com/115425/', 'https://www.ethiopianreporter.com/115423/', 'http://www.ethiopianreporter.com/wp-content/uploads/2023/01/fermataa.

In [7]:
# <tbody>
#  <tr>
#   <td>
#     <a href="https://www.ethiopianreporter.com/post-sitemap1.xml">https://www.ethiopianreporter.com/post-sitemap1.xml</a>

In [8]:
# how many urls?
len(list_of_URLS)


28915

In [9]:
# get rid of duplicates
unique_list = list(set(list_of_URLS))
print(len(unique_list))


28215


In [10]:
# set name again
scrapedURLS = unique_list


In [11]:
# an example url
print(scrapedURLS[0])


https://www.ethiopianreporter.com/72603/


In [12]:
# adapted from https://blog.devgenius.io/use-selenium-webdriver-in-google-colab-d5f2dba1d9f5
#First, we install cloudscraper and googletrans modules that we are going to use for scraping and translating the content.

!pip install cloudscraper
!pip install google_trans_new


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 7.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
#With cloudscraper we scrape the URL and with Beutiful Soup we parse the URLs.
#We put everything together in order based on importance.
import cloudscraper
from bs4 import BeautifulSoup


In [17]:
# declare data as an empty DataFrame
data = pd.DataFrame(columns = ['Date', 'Title', 'URL'])

def scrape_this_url(url):
    scraper = cloudscraper.create_scraper() 
    headers = {'user-agent': 'Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}
    
    try:
        r = scraper.get(url, headers=headers)
        soup = BeautifulSoup(r.content, 'lxml')
        # get title, date
        title = soup.find("title").text # Extract the text content of the title tag
        hyphen_index = title.find(" - ") # Find the index of the hyphen that separates the unwanted words from the rest of the title
        if hyphen_index != -1: # If the hyphen was found
            title = title[:hyphen_index] # Extract the substring of the title up to the hyphen
        time_tag = soup.find('time')
        if time_tag:
            date = time_tag['datetime'].split('T')[0] # Extract the date part from the datetime value
        else:
            date = ""

        # Create a dictionary with the data to add
        data_dict = {'Date': date, 'Title': title, 'URL': url}


        # return the dictionary
        return data_dict

    except Exception as e:
        print(e)
        return {}


In [15]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")


In [19]:
# loop through the list of URLs and append the data to the DataFrame
# it's capped to 10 URLS for now.
for url in scrapedURLS[:10]:
    data_dict = scrape_this_url(url)
    if data_dict:
        data = data.append(data_dict, ignore_index=True)


In [20]:
data.head()


,Date,Title,URL
0,2021-01-31,ተስፍሽ እና ገብርሽ,https://www.ethiopianreporter.com/72603/
1,2015-09-16,የቀላል ባቡር አገልግሎት ደኅንነትን ለመጠበቅ ብሔራዊ መረጃና ደኅንነት የ...,https://www.ethiopianreporter.com/53776/
2,2016-07-06,በፖላንዱ የዓለም ወጣቶች ሻምፒዮና 25 አትሌቶች ይሳተፋሉ,https://www.ethiopianreporter.com/50318/
3,2020-08-09,ሰባኛ ዓመቱ እየታሰበ ያለው ቃኘው የከተተበት የኮርያ ዘመቻ,https://www.ethiopianreporter.com/70144/
4,2018-03-28,የምን መፍረክረክ ነው?,https://www.ethiopianreporter.com/54914/


In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
from google.colab import files
data.to_csv('output.csv', encoding = 'utf-8-sig') 
files.download('output.csv')